Загрузка библиотек

In [3]:
pip install transformers datasets evaluate accelerate

In [4]:
from datasets import load_dataset
from transformers import AutoTokenizer
from transformers import DataCollatorWithPadding
import evaluate
import numpy as np
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
from transformers import EarlyStoppingCallback
from transformers import pipeline
from transformers.pipelines.pt_utils import KeyDataset
import numpy as np
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import classification_report

Загрузка датасета

In [5]:
dataset = load_dataset('imdb', trust_remote_code=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Обучение

In [6]:
def get_tokenizer(examples):
    return tokenizer(examples['text'], truncation=True)

In [7]:
tokenizer = AutoTokenizer.from_pretrained('gpt2')


tokenized_dataset = dataset.map(get_tokenizer, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [8]:
accuracy = evaluate.load('accuracy')

In [9]:
id_key = {0: 'neg', 1: 'pos'}
label_key = {'neg': 0, 'pos': 1}

In [10]:
model = AutoModelForSequenceClassification.from_pretrained('gpt2', num_labels=2, id2label=id_key, label2id=label_key)

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
def get_acc(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [12]:
training_args = TrainingArguments(output_dir='results', learning_rate=1e-6, per_device_train_batch_size=1, per_device_eval_batch_size=1, num_train_epochs=1, weight_decay=0.1, eval_strategy='epoch', save_strategy='epoch', save_total_limit=4, load_best_model_at_end=True)

In [14]:
tokenizer.pad_token = tokenizer.eos_token

In [15]:
trainer = Trainer(model=model, args=training_args, train_dataset=tokenized_dataset["train"], tokenizer=tokenizer, data_collator=data_collator, compute_metrics=get_acc, callbacks=[EarlyStoppingCallback(early_stopping_patience=4)])
trainer.train()

Epoch,Training Loss,Validation Loss


Epoch,Training Loss,Validation Loss


ValueError: Trainer: evaluation requires an eval_dataset.

In [25]:
trainer.test_dataset = tokenized_dataset['test']

In [26]:
eval_results = trainer.evaluate(test_dataset)

Epoch,Training Loss,Validation Loss,Accuracy
1,0.778000,0.696464,0.913400
